In [1]:
# Loading all the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.offsetbox import AnchoredText
from urllib.parse import unquote
import seaborn as sns
import networkx as nx
#import plotly.express as px
#import plotly.graph_objects as go
import statsmodels.formula.api as smf
import scipy.stats as stats

In [2]:
articles =  pd.read_csv("/Users/zainebtafasca/Desktop/ADA/P2 ADA/wikispeedia_paths-and-graph/articles.tsv", sep='\t', names=["article"], skiprows=12)

In [3]:
links = pd.read_csv("/Users/zainebtafasca/Desktop/ADA/P2 ADA/wikispeedia_paths-and-graph/links.tsv", sep='\t', names=["link_source", "link_target"], skiprows = 12)
paths_finished = pd.read_csv("/Users/zainebtafasca/Desktop/ADA/P2 ADA/wikispeedia_paths-and-graph/paths_finished.tsv", sep='\t', names=["hashedIpAddress", "timestamp", 
                                                              "durationInSec", "path", "rating"], skiprows = 15)
paths_unfinished = pd.read_csv("/Users/zainebtafasca/Desktop/ADA/P2 ADA/wikispeedia_paths-and-graph/paths_unfinished.tsv", sep='\t', names=["hashedIpAddress", "timestamp", 
                                                                  "durationInSec", "path", "target", "type"], skiprows = 17)
shortest_distance_text = np.loadtxt("/Users/zainebtafasca/Desktop/ADA/P2 ADA/wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt", dtype=str, comments="#", unpack=False)
categories= pd.read_csv("/Users/zainebtafasca/Desktop/ADA/P2 ADA/wikispeedia_paths-and-graph/categories.tsv", sep='\t', names=["article", "category"], skiprows = 13)

In [4]:
print(shortest_distance_text[:2])

['0_____33333325634333435_2433544334_3_42234354456642455553533242_4_33433_43_3343_34__333232424334_344433343444_344442_3334_3_334324__453_4_4344443434323333344444343_43334_344_33334_33343333434_343__3444443433433435433343_33333344534433344343322232444_34444444533323432343344433_2343443_423323_43_43_544334333433332_34434333__2333444335343334333_43333_4_342443_44343433_4_43333352_343334433444332343__3343_34_34_354_43433_43333424343443_343_23_33_233333344333_2334244333243363334_55344_4343634333453443333433333464333434443_432334341433_24444333222__32_333334__4344332433_42335333344333333_3334333333334_534_3_3374_55434333_354_43434443344434344_4334343533244343344_33434_355_44443353333343_3332323222222433233_323344_33433343434334_33424334_4343434433343335434445223333243333333433443333433_33234_43433353233_334434_344433434544___432225433423334434443452343353343444432333534_3344455434433433_32_354333433432433345322_3343332_23344433323_332334332_34443344324333_4443343_445145445354_444_5_35_4_442

In [5]:
# seperate each row and create a matrix with all the shortest distance

table_distance = np.array([[int(c) if c != '_' else -1 for c in row] for row in shortest_distance_text])

table_distance[0]




array([ 0, -1, -1, ...,  4,  4,  2])

In [6]:
def seperate_path(path):

    # Split the path into individual articles
    articles = path.split(";")
    
    # Extract source and target
    source, target = articles[0], articles[-1]
    
    # Calculate the length, considering back-clicks
    length = len(articles) + articles.count('') - 1
    
    return source, target, length



In [7]:
# expand the finished paths
path_fin = paths_finished.copy()
path_fin = path_fin.assign(source=paths_finished['path'].apply(lambda x: seperate_path(x)[0]),
                           target=paths_finished['path'].apply(lambda x: seperate_path(x)[1]),
                           game_path=paths_finished['path'].apply(lambda x: seperate_path(x)[2]),
                           completed=True)

# expand the unfinished paths
path_ufin = paths_unfinished.copy()
path_ufin = path_ufin.assign(source=paths_unfinished['path'].apply(lambda x: seperate_path(x)[0]),
                             failure_article=paths_unfinished['path'].apply(lambda x: seperate_path(x)[1]),
                             game_path=paths_unfinished['path'].apply(lambda x: seperate_path(x)[2]),
                             completed=False,
                             target=paths_unfinished["target"])

# construct a new dataframe: path_overview
path_overview = pd.concat([path_fin, path_ufin], ignore_index=True)[["source", "target",  "hashedIpAddress", "timestamp",
                                                                     "durationInSec", "game_path",
                                                                     "completed", "rating","type"]]
                                                                       



In [8]:
# Find the index of an article in the helper DataFrame
def find_article_index(article, helper_df):
    try:
        return helper_df.loc[article]
    except KeyError:
        return None

# Calculate the shortest distance between source and target indices
def calculate_shortest_distance(source_idx, target_idx):
    try:
        return table_distance[source_idx, target_idx][0]
    except KeyError:
        return None

# Finding the shortest path length between a source and a target
def find_path_distance(source, target, helper_df):
    source_idx = find_article_index(source, helper_df)
    target_idx = find_article_index(target, helper_df)

    if source_idx is not None and target_idx is not None:
        return calculate_shortest_distance(source_idx, target_idx)
    else:
        # Handle the case where source or target is not present in the index
        return None  # You can customize this behavior based on your specific needs



In [9]:
articles["index"] = articles.index
help_articles = articles.set_index("article")

# shortest path length is added to the path_overview
# Create a copy of the 'path_overview' DataFrame to avoid modifying the original
path_overview_copy = path_overview.copy()

# Apply the find_path_distance function to each row
path_overview_copy['shortest_distance'] = path_overview_copy.apply(
    lambda row: find_path_distance(row['source'], row['target'], help_articles), axis=1
)

# Update the original 'path_overview' DataFrame with the new 'shortest_distance' column
path_overview['shortest_distance'] = path_overview_copy['shortest_distance']



In [10]:

#remove all rows where the source or target article is not among the available articles in the dataset
path_overview = path_overview.drop(path_overview[~(path_overview['source'].isin(articles['article'].array))].index)
path_overview = path_overview.drop(path_overview[~(path_overview['target'].isin(articles['article'].array))].index)

# remove all rows where there is no path between the source article and target article
path_overview = path_overview.drop(path_overview[path_overview['shortest_distance'] == -1].index) 
path_overview.insert(4, 'shortest_distance', path_overview.pop('shortest_distance'))
path_overview

,source,target,hashedIpAddress,timestamp,shortest_distance,durationInSec,game_path,completed,rating,type
0,14th_century,African_slave_trade,6a3701d319fc3754,1297740409,3.0,166,8,True,NaN,NaN
1,14th_century,African_slave_trade,3824310e536af032,1344753412,3.0,88,4,True,3.0,NaN
2,14th_century,African_slave_trade,415612e93584d30e,1349298640,3.0,138,7,True,NaN,NaN
3,14th_century,Greece,64dd5cd342e3780c,1265613925,2.0,37,3,True,NaN,NaN
4,14th_century,John_F._Kennedy,015245d773376aab,1366730828,3.0,175,6,True,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...
76188,Franz_Kafka,Cholera,109ed71f571d86e9,1389787605,3.0,180,7,False,NaN,restart
76189,Modern_history,Hollandic,232f992e57d43e8d,1389787697,3.0,6,0,False,NaN,restart
76190,Computer_programming,The_Beatles,2e09a7224600a7cd,1389798400,3.0,1900,4,False,NaN,timeout
76191,Jamaica,Alan_Turing,60af9e2138051b96,1389799481,3.0,1903,3,False,NaN,timeout


In [ ]:
# extract categories
def extract_subject_category(category):
    # Split the category string by '.'
    category_parts = category.split('.')
    
    # Find the index of 'subject' in the split parts
    subject_index = category_parts.index('subject')
    
    # Check if there is at least one word after 'subject'
    if subject_index < len(category_parts) - 1:
        return category_parts[subject_index + 1]

# Apply the custom function to create a new 'subject_category' column
categories['subject_category'] = categories['category'].apply(extract_subject_category)

# Create a new DataFrame with only the relevant columns
new_dataframe = categories[['article', 'subject_category']]

# Display the new DataFrame
print(new_dataframe.head(20))


